Check the files imported using the UI

In [0]:
dbutils.fs.ls('/FileStore/tables')

Out[29]: [FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1645025994000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019/', name='clinicaltrial_2019/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.csv', name='clinicaltrial_2019.csv', size=42400056, modificationTime=1648298404000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020.csv', name='clinicaltrial_2020.csv', size=46318151, modificationTime=1648298480000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021.csv', name='clinicaltrial_2021.csv', size=50359696, modificationTime=1648298499000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021_csv_gz.gz', name='clinicaltrial_2021_csv_gz.gz', size=11921810, modificationTime=1651935392000),
 FileInfo(path='dbfs:/FileStore/tables/data_scrubbing_using_rdds.ipynb', name='data_scrubbing_using_rdds.ipynb', size=16994, modificationTime=1645531555000),
 FileInfo

Move the files from fileStore/table to tmp folder

In [0]:
dbutils.fs.mv('/FileStore/tables/clinicaltrial_2021_csv.gz' , 'file:/tmp/')

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-4296408711223240> in <module>
----> 1 dbutils.fs.mv('/FileStore/tables/clinicaltrial_2021_csv.gz' , 'file:/tmp/')

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    387                     exc.__context__ = None
    388                     exc.__cause__ = None
--> 389                     raise exc
    390 
    391             return f_with_exception_handling

ExecutionError: An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.mv.
: java.io.FileNotFoundException: /FileStore/tables/clinicaltrial_2021_csv.gz
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.getFile

Unzip the gz files

In [0]:
%sh
gunzip -d /tmp/ /tmp/clinicaltrial_2021_csv.gz
ls

Move the unzipped files from tmp folder back to DBFS

In [0]:
dbutils.fs.mv('file:/tmp/clinicaltrial_2021_csv' , '/FileStore/tables/clinicaltrial_2021.csv' , True)

In [0]:
dbutils.fs.ls('/FileStore/tables/')

QUESTION 1 (THE NUMBER OF DISTINCT STUDIES IN THE DATASET)

In [0]:
ClinicalRDD = sc.textFile('/FileStore/tables/clinicaltrial_2021.csv')

In [0]:
ClinicalRDD.take(5)

Out[32]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|']

Remove the header row

In [0]:
header_row= ClinicalRDD.first()
ClinicalRDD2 = ClinicalRDD.filter(lambda row: row !=header_row)

In [0]:
ClinicalRDD2.take(5)

Out[34]: ['NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|',
 'NCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|']

In [0]:
ClinicalRDD2.distinct().count()

Out[35]: 387261

QUESTION 2 (LIST ALL THE TYPES AS CONTAINED IN THE TYPE COLUMN ARRAGED IN DESCENDING ORDER)

In [0]:
TypeRDD= ClinicalRDD2.map(lambda x: x.split("|"))
TypeRDD.take(5)

Out[36]: [['NCT02758028',
  'The University of Hong Kong',
  'Recruiting',
  'Aug 2005',
  'Nov 2021',
  'Interventional',
  'Apr 2016',
  '',
  ''],
 ['NCT02751957',
  'Duke University',
  'Completed',
  'Jul 2016',
  'Jul 2020',
  'Interventional',
  'Apr 2016',
  'Autistic Disorder,Autism Spectrum Disorder',
  ''],
 ['NCT02758483',
  'Universidade Federal do Rio de Janeiro',
  'Completed',
  'Mar 2017',
  'Jan 2018',
  'Interventional',
  'Apr 2016',
  'Diabetes Mellitus',
  ''],
 ['NCT02759848',
  'Istanbul Medeniyet University',
  'Completed',
  'Jan 2012',
  'Dec 2014',
  'Observational',
  'May 2016',
  'Tuberculosis,Lung Diseases,Pulmonary Disease',
  ''],
 ['NCT02758860',
  'University of Roma La Sapienza',
  'Active, not recruiting',
  'Jun 2016',
  'Sep 2020',
  'Observational [Patient Registry]',
  'Apr 2016',
  'Diverticular Diseases,Diverticulum,Diverticulosis',
  '']]

In [0]:
TypeRDD2 = TypeRDD.map(lambda a:(a[5],1))
TypeRDD2.collect()

Out[37]: [('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Observational', 1),
 ('Observational [Patient Registry]', 1),
 ('Interventional', 1),
 ('Observational [Patient Registry]', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Observational', 1),
 ('Observational', 1),
 ('Observational', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Observational', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Observational', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Observational', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 ('Interventional', 1),
 (

In [0]:
All_Types=TypeRDD2.reduceByKey(lambda x1,x2: x1+x2).sortBy(lambda s: s[1], ascending =False)
All_Types.collect()

Out[38]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

QUESTION 3 (LIST TE TOP FIV CONDITIONS AND THEIR CORESPONDING FREQUENCIES)

In [0]:
Condition_Column = TypeRDD.map(lambda a:(a[7]))
Condition_Column.take(10)

Out[39]: ['',
 'Autistic Disorder,Autism Spectrum Disorder',
 'Diabetes Mellitus',
 'Tuberculosis,Lung Diseases,Pulmonary Disease',
 'Diverticular Diseases,Diverticulum,Diverticulosis',
 'Asthma',
 'Hypoventilation',
 'Lymphoma',
 '',
 'Myositis']

In [0]:
Removespaces = ['']
Emptyspaces = Condition_Column.filter(lambda line:line not in Removespaces)
Emptyspaces.collect()

Out[40]: ['Autistic Disorder,Autism Spectrum Disorder',
 'Diabetes Mellitus',
 'Tuberculosis,Lung Diseases,Pulmonary Disease',
 'Diverticular Diseases,Diverticulum,Diverticulosis',
 'Asthma',
 'Hypoventilation',
 'Lymphoma',
 'Myositis',
 'Diabetes Mellitus',
 'Hypertension',
 'Periodontal Diseases',
 'Diabetes Mellitus',
 'Appendicitis,Stomach Ulcer,Cholecystolithiasis,Cholelithiasis,Gallstones',
 'Hookworm Infections,Celiac Disease',
 'Ascites',
 'Chronic Pain,Substance-Related Disorders,Opioid-Related Disorders,Narcotic-Related Disorders,Behavior',
 'Sleep Apnea Syndromes,Sleep Apnea',
 'Tyrosinemias',
 'Hemangioma',
 'Hypertension',
 'Carcinoma',
 'Lymphoma',
 'Iatrogenic Disease',
 'Cystic Fibrosis,Fibrosis',
 'Motor Neuron Disease,Amyotrophic Lateral Sclerosis,Sclerosis',
 'Depressive Disorder,Depression',
 'Schizophrenia',
 'Myocardial Infarction,Cardiomyopathies,Microvascular Angina,Takotsubo Cardiomyopathy,Syndrome,Infarction',
 'Cystic Fibrosis,Fibrosis',
 'Hyperphagia',
 'Ac

In [0]:
Conditions = Emptyspaces.flatMap(lambda s:(s.split(","))).map(lambda a:(a, 1))
Conditions.collect()

Out[41]: [('Autistic Disorder', 1),
 ('Autism Spectrum Disorder', 1),
 ('Diabetes Mellitus', 1),
 ('Tuberculosis', 1),
 ('Lung Diseases', 1),
 ('Pulmonary Disease', 1),
 ('Diverticular Diseases', 1),
 ('Diverticulum', 1),
 ('Diverticulosis', 1),
 ('Asthma', 1),
 ('Hypoventilation', 1),
 ('Lymphoma', 1),
 ('Myositis', 1),
 ('Diabetes Mellitus', 1),
 ('Hypertension', 1),
 ('Periodontal Diseases', 1),
 ('Diabetes Mellitus', 1),
 ('Appendicitis', 1),
 ('Stomach Ulcer', 1),
 ('Cholecystolithiasis', 1),
 ('Cholelithiasis', 1),
 ('Gallstones', 1),
 ('Hookworm Infections', 1),
 ('Celiac Disease', 1),
 ('Ascites', 1),
 ('Chronic Pain', 1),
 ('Substance-Related Disorders', 1),
 ('Opioid-Related Disorders', 1),
 ('Narcotic-Related Disorders', 1),
 ('Behavior', 1),
 ('Sleep Apnea Syndromes', 1),
 ('Sleep Apnea', 1),
 ('Tyrosinemias', 1),
 ('Hemangioma', 1),
 ('Hypertension', 1),
 ('Carcinoma', 1),
 ('Lymphoma', 1),
 ('Iatrogenic Disease', 1),
 ('Cystic Fibrosis', 1),
 ('Fibrosis', 1),
 ('Motor Neu

In [0]:
Top5_Conditions= Conditions.reduceByKey(lambda x1,x2: x1+x2).sortBy(lambda s: s[1], ascending =False)
Top5_Conditions.take(5)

Out[42]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

QUESTION 4

In [0]:
MeshRDD = sc.textFile('/FileStore/tables/mesh.csv')

In [0]:
MeshRDD.take(20)

Out[44]: ['term,tree',
 'Calcimycin,D03.633.100.221.173',
 'A-23187,D03.633.100.221.173',
 'Temefos,D02.705.400.625.800',
 'Temefos,D02.705.539.345.800',
 'Temefos,D02.886.300.692.800',
 'Abate,D02.705.400.625.800',
 'Abate,D02.705.539.345.800',
 'Abate,D02.886.300.692.800',
 'Difos,D02.705.400.625.800',
 'Difos,D02.705.539.345.800',
 'Difos,D02.886.300.692.800',
 'Abattoirs,J01.576.423.200.700.100',
 'Abattoirs,J03.540.020',
 'Abbreviations as Topic,L01.559.598.400.556.131',
 'Acronyms as Topic,L01.559.598.400.556.131',
 'Abdomen,A01.923.047',
 '"Abdomen, Acute",C23.888.592.612.054.200',
 '"Abdomen, Acute",C23.888.821.030.249',
 'Abdominal Injuries,C26.017']

In [0]:
header_row= MeshRDD.first()
MeshRDD2 = MeshRDD.filter(lambda row: row !=header_row)

In [0]:
MeshRDD3 = MeshRDD2.map(lambda a:(a.split(",")))
MeshRDD3.take(50)

Out[46]: [['Calcimycin', 'D03.633.100.221.173'],
 ['A-23187', 'D03.633.100.221.173'],
 ['Temefos', 'D02.705.400.625.800'],
 ['Temefos', 'D02.705.539.345.800'],
 ['Temefos', 'D02.886.300.692.800'],
 ['Abate', 'D02.705.400.625.800'],
 ['Abate', 'D02.705.539.345.800'],
 ['Abate', 'D02.886.300.692.800'],
 ['Difos', 'D02.705.400.625.800'],
 ['Difos', 'D02.705.539.345.800'],
 ['Difos', 'D02.886.300.692.800'],
 ['Abattoirs', 'J01.576.423.200.700.100'],
 ['Abattoirs', 'J03.540.020'],
 ['Abbreviations as Topic', 'L01.559.598.400.556.131'],
 ['Acronyms as Topic', 'L01.559.598.400.556.131'],
 ['Abdomen', 'A01.923.047'],
 ['"Abdomen', ' Acute"', 'C23.888.592.612.054.200'],
 ['"Abdomen', ' Acute"', 'C23.888.821.030.249'],
 ['Abdominal Injuries', 'C26.017'],
 ['Abdominal Neoplasms', 'C04.588.033'],
 ['Abdominal Muscles', 'A02.633.567.050'],
 ['Cremaster Muscle', 'A02.633.567.050'],
 ['Pyramidalis Muscle', 'A02.633.567.050'],
 ['Quadratus Abdominis', 'A02.633.567.050'],
 ['Transversus Abdominis', 'A0

In [0]:
Combined_Clin_mesh = Conditions.join(MeshRDD3)
Combined_Clin_mesh.take(50)

Out[47]: [('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('

In [0]:
Q4RDD = Combined_Clin_mesh.map(lambda x:(x[1])).map(lambda x: (x[1], x[0])).map(lambda x: (x[0].split(".")[0],x[1])).reduceByKey(lambda x1,x2: x1+x2).sortBy(lambda x : x[1], ascending = False)
Q4RDD.take(5)

Out[48]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

QUESTION 5

In [0]:
PharmaRDD = sc.textFile('/FileStore/tables/pharma.csv')
PharmaRDD.take(10)

Out[49]: ['"Company","Parent_Company","Penalty_Amount","Subtraction_From_Penalty","Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting","Penalty_Year","Penalty_Date","Offense_Group","Primary_Offense","Secondary_Offense","Description","Level_of_Government","Action_Type","Agency","Civil/Criminal","Prosecution_Agreement","Court","Case_ID","Private_Litigation_Case_Title","Lawsuit_Resolution","Facility_State","City","Address","Zip","NAICS_Code","NAICS_Translation","HQ_Country_of_Parent","HQ_State_of_Parent","Ownership_Structure","Parent_Company_Stock_Ticker","Major_Industry_of_Parent","Specific_Industry_of_Parent","Info_Source","Notes"',
 '"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the c

In [0]:
header_row= PharmaRDD.first()
PharmaRDD2 = PharmaRDD.filter(lambda row: row !=header_row)

In [0]:
PharmaRDD2.take(10)

Out[51]: ['"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the company\'s carotid, biliary and peripheral vascular products.","federal","agency action","Justice Department Civil Division","civil","","","","","","","","","","","","USA","Illinois","publicly traded","ABT","pharmaceuticals","pharmaceuticals","https://www.justice.gov/opa/pr/abbott-laboratories-pays-us-5475-million-settle-claims-company-paid-kickbacks-physicians",""',
 '"Abbott Laboratories Inc.","AbbVie","$1,500,000,000","$0","$1,500,000,000","2012","20120507","healthcare-related offenses","off-label or unapproved promotion of medical products","","Global Health Care Company Abbott Laboratories Inc. has pleaded guilty and agreed to

In [0]:
PharmaRDD3 = PharmaRDD2.map(lambda a: a.split('","'))
PharmaRDD3.take(5)

Out[52]: [['"Abbott Laboratories',
  'Abbott Laboratories',
  '$5,475,000',
  '$0',
  '$5,475,000',
  '2013',
  '20131227',
  'government-contracting-related offenses',
  'False Claims Act and related',
  'kickbacks and bribery',
  "Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the company's carotid, biliary and peripheral vascular products.",
  'federal',
  'agency action',
  'Justice Department Civil Division',
  'civil',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'USA',
  'Illinois',
  'publicly traded',
  'ABT',
  'pharmaceuticals',
  'pharmaceuticals',
  'https://www.justice.gov/opa/pr/abbott-laboratories-pays-us-5475-million-settle-claims-company-paid-kickbacks-physicians',
  '"'],
 ['"Abbott Laboratories Inc.',
  'AbbVie',
  '$1,500,000,000',
  '$0',
  '$1,500,000,000',
  '2012',
  '20120507',
  'healthcare-related offenses',
  'off-label or unapp

In [0]:
Pharmacitical = PharmaRDD3.map(lambda x: (x[1],1))
Pharmacitical.collect()

Out[53]: [('Abbott Laboratories', 1),
 ('AbbVie', 1),
 ('AbbVie', 1),
 ('Abbott Laboratories', 1),
 ('Johnson & Johnson', 1),
 ('Abbott Laboratories', 1),
 ('Abbott Laboratories', 1),
 ('Johnson & Johnson', 1),
 ('Johnson & Johnson', 1),
 ('Abbott Laboratories', 1),
 ('AbbVie', 1),
 ('AbbVie', 1),
 ('Pfizer', 1),
 ('Pfizer', 1),
 ('Bristol-Myers Squibb', 1),
 ('Amgen', 1),
 ('Amgen', 1),
 ('Amgen', 1),
 ('Amneal Pharmaceuticals', 1),
 ('Astellas Pharma', 1),
 ('AstraZeneca', 1),
 ('AstraZeneca', 1),
 ('AstraZeneca', 1),
 ('AstraZeneca', 1),
 ('AstraZeneca', 1),
 ('AstraZeneca', 1),
 ('AstraZeneca', 1),
 ('Sanofi', 1),
 ('Sanofi', 1),
 ('AVEO Pharmaceuticals', 1),
 ('Biogen Idec', 1),
 ('Biogen Idec', 1),
 ('Bausch Health', 1),
 ('Bausch Health', 1),
 ('GlaxoSmithKline', 1),
 ('Boehringer Ingelheim', 1),
 ('Boehringer Ingelheim', 1),
 ('Eli Lilly', 1),
 ('Eli Lilly', 1),
 ('Bristol-Myers Squibb', 1),
 ('Bristol-Myers Squibb', 1),
 ('Bristol-Myers Squibb', 1),
 ('Bristol-Myers Squibb', 1

In [0]:
Sponsor = TypeRDD.map(lambda x: (x[1],1))
Sponsor.collect()

Out[54]: [('Sponsor', 1),
 ('The University of Hong Kong', 1),
 ('Duke University', 1),
 ('Universidade Federal do Rio de Janeiro', 1),
 ('Istanbul Medeniyet University', 1),
 ('University of Roma La Sapienza', 1),
 ('Consorzio Futuro in Ricerca', 1),
 ('Ankara University', 1),
 ('Ruijin Hospital', 1),
 ('Washington University School of Medicine', 1),
 ('Orphazyme', 1),
 ('Novo Nordisk A/S', 1),
 ('Daniel Alexandre Bottino', 1),
 ('Bulent Ecevit University', 1),
 ('Institut für Pharmakologie und Präventive Medizin', 1),
 ('The Third Xiangya Hospital of Central South University', 1),
 ('Tel Aviv Medical Center', 1),
 ('Medicines for Malaria Venture', 1),
 ('James Cook University, Queensland, Australia', 1),
 ('Soonchunhyang University Hospital', 1),
 ('Member Companies of the Opioid PMR Consortium', 1),
 ('Marmara University', 1),
 ('Cycle Pharmaceuticals Ltd.', 1),
 ('Yonsei University', 1),
 ('Cutera Inc.', 1),
 ('University of Zurich', 1),
 ('Guangdong Association of Clinical Trials'

In [0]:
Not_Pharmaceutical = Sponsor.leftOuterJoin(Pharmacitical).subtractByKey(Pharmacitical).map(lambda x: x[0]).flatMap(lambda x: [(x,1)]).reduceByKey(lambda x , y: x+y).sortBy(lambda x:x[1], ascending = False)
Not_Pharmaceutical.take(10)

Out[55]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

QUESTION 6

In [0]:
Months_Completed_Studies =  TypeRDD.map(lambda x: (x[2],x[4])).filter(lambda x: '2021' in x[1]).filter(lambda x: 'Completed' in x[0])
Months_Completed_Studies.take(15)

Out[56]: [('Completed', 'Jan 2021'),
 ('Completed', 'Jun 2021'),
 ('Completed', 'Mar 2021'),
 ('Completed', 'Jan 2021'),
 ('Completed', 'May 2021'),
 ('Completed', 'Feb 2021'),
 ('Completed', 'Feb 2021'),
 ('Completed', 'May 2021'),
 ('Completed', 'Aug 2021'),
 ('Completed', 'May 2021'),
 ('Completed', 'Mar 2021'),
 ('Completed', 'Feb 2021'),
 ('Completed', 'May 2021'),
 ('Completed', 'May 2021'),
 ('Completed', 'May 2021')]

In [0]:
Months_Completed_Studies2 = Months_Completed_Studies.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x+y).map(lambda x : (x[0].split( )[0],x[1]))
Months_Completed_Studies2.take(12)

Out[57]: [('Jan', 1131),
 ('Jun', 1094),
 ('Aug', 700),
 ('Apr', 967),
 ('Mar', 1227),
 ('May', 984),
 ('Feb', 934),
 ('Jul', 819),
 ('Oct', 187),
 ('Sep', 528)]

In [0]:
import calendar
year = {i:e for e, i in enumerate(calendar.month_abbr[1:],1)}
Months_Completed_Studies2.sortBy(keyfunc = lambda x : year.get(x[0])).collect()

Out[58]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]